# LLM Zoomcamp 2024 - Session #1 - Homework

Author: José Victor

## Q1. Running Elastic

Run Elastic Search 8.4.3, and get the clust information. If you run it on localhost, this is how you do it:

```bash
curl localhost:9200
```

What's the `version.build_hash` value?

## Getting the data

Now let's get the FAQ data. You can run this snippet:

```python
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
```

Note that you need to have the `requests` library:

```bash
pip install requests
```

In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
!curl localhost:9200

{
  "name" : "0864890e2664",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "EfTUlTsxSP-yn0CYWoI1BA",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


Q1 answer

`version.build_hash`: "42f05b9372a9a4a470db3b52817899b99a76ee73"

## Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the `course` field a keyword and the rest should be text.

Don't forget to install the ElasticSearch client for Python:

```bash
pip install elasticsearch
```

Which function do you use for adding your data to elastic?

* ( ) `insert`
* (X) `index`
* ( ) `put`
* ( ) `add`  

In [3]:
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
es_client = Elasticsearch("http://localhost:9200")

In [5]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [6]:
for doc in tqdm(documents): 
    es_client.index(index=index_name, document=doc)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:23<00:00, 41.01it/s]


## Q3. Searching

Now let's search in our index.

For a query "How do I execute a command in a running docker container?", what's the score for the top ranking result?

Use only `question` and `text` fields and give `question` a boost of 4

* ( ) 94.05
* (X) 84.05
* ( ) 74.05
* ( ) 64.05

Look at the `_score` field.

In [7]:
query = "How do I execute a command in a running docker container?"

In [8]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
        }
    }
}

In [9]:
response = es_client.search(index=index_name, body=search_query)

In [10]:
ranking_results = sorted([hit['_score'] for hit in response['hits']['hits']], reverse=True)
top_score = ranking_results[0]
print(top_score)

84.050095


## Q4. Filtering

Now let's only limit the question to `machine-learning-zoomcamp`.

Return 3 results. What's the 3rd question returned by the search engine?

* ( ) How do I debug a docker container?
* (X) How do I copy files from a different folder into docker container's working directory?
* ( ) How do Lambda container images work?
* ( ) How can I annotate a graph?

In [11]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

In [12]:
response = es_client.search(index=index_name, body=search_query)

In [13]:
questions_list = [hit['_source'] for hit in response['hits']['hits']]
print(questions_list[2]["question"])

How do I copy files from a different folder into docker container’s working directory?


## Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM. Let's use these templates:

```python
context_template = """
Q: {question}
A: {text}
""".strip()

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```

Separate context entries by two linebreaks (`\n\n`)

Put the data from Q4 there.

What's the length of the result? (use the `len` function)

* ( ) 962
* (X) 1462
* ( ) 1962
* ( ) 2462

In [39]:
context_template = """
Q: {question}
A: {text}
""".strip()

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [44]:
context = ""

for i in range(len(questions_list)):
    context = context + context_template.format(question=questions_list[i]["question"], text=questions_list[i]["text"]) + "\n\n"

prompt = prompt_template.format(question=questions_list[2]["question"], context=context).strip()

In [45]:
len(prompt)

1491

## Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of tokens we send in our prompt and receive in the response.

The OpenAI python package uses `tiktoken` for tokenization:

```bash
pip install tiktoken
```

Let's calculate the number of tokens in our query:

```python
encoding = tiktoken.encoding_for_model("gpt-4o")
``` 
Use the `encode` function. How many tokens does our prompt have?

* ( ) 122
* ( ) 222
* (X) 322
* ( ) 422

Note: to decode back a token into a word, you can use the `decode_single_token_bytes` function:

```python
encoding.decode_single_token_bytes(63842)
```

In [31]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.3 MB/s eta 0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.1/775.1 kB 18.0 MB/s eta 0:00:00m eta 0:00:01


In [33]:
import tiktoken

In [34]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [38]:
len(encoding.encode(prompt))

326

## Bonus: generating the answer (ungraded)

Let's send the prompt to OpenAI. What's the response?

Note: you can replace OpenAI with Ollama. See module 2.

## Bonus: calculating the costs (ungraded)

Suppose that on average per request we send 150 tokens and receive back 250 tokens.

How much will it cost to run 1000 requests?

You can see the prices [here](https://openai.com/api/pricing/)

* Input: $ 0.005 / 1K tokens
  
* Output: $ 0.015 / 1K tokens

You can redo the calculations with the values you got in Q6 and Q7.